In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Estimación del variograma direccional
<img src="data/variograma_direccional.png" width="500">

- $\delta$ >> Angular tolerance
- $t_{h}$ >> Lag tolerance
- $t_{b}$ >> Bandwidth. 

In [ ]:
# Importamos módulos para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstools as gs

# Posibles modelos de variograma
models = {
    "Gaussian": gs.Gaussian,
    "Exponential": gs.Exponential,
    "Stable": gs.Stable,
    "Rational": gs.Rational,
    "Circular": gs.Circular,
    "Spherical": gs.Spherical,
    "SuperSpherical": gs.SuperSpherical,
    "JBessel": gs.JBessel,
}

In [ ]:
# ESPEFICICAMOS ARCHIVO Y SEPARADOR DE COLUMNAS
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
FILE = "data/UNCF_dataset.xlsx"
HOJA = "Hoja1"
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
data = pd.read_excel(FILE, HOJA, engine = "openpyxl")
data.describe()

In [ ]:
# ESPEFICICAMOS CAMPOS Y REPRESENTAMOS
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
XCOL = "X"
YCOL = "Y"
VCOL = "Elevation (ft)"
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

xi = data[XCOL]
yi = data[YCOL]
val = data[VCOL]*-1
plt.scatter(xi, yi, marker="s")

In [ ]:
# ESTIMACIÓN DEL VARIOGRAMA EXPERIMENTAL
# EN 2 DIRECCIONES
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
bins = None
#bins = np.arange(400, 11000, 600)
ANGULOS = [0, 90]
ANG_TOL = 22.5
BANDWIDTH = None
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

coords = [xi, yi]
angles = np.deg2rad(ANGULOS)
ang_tol = np.deg2rad(ANG_TOL)
bin_center, gamma = gs.vario_estimate(coords, val, bins, angles=angles, angles_tol=ang_tol, bandwidth=BANDWIDTH)
fig = plt.figure()
ax = fig.add_subplot(111)

for n in range(angles.size):
    ax.plot(bin_center, gamma[n], label="{}º".format(ANGULOS[n]), marker="o", ls="--")
    
ax.set_ylabel("$\gamma$")
res = ax.set_xlabel("Distance")
ax.legend()

In [ ]:
# AJUSTE AUTOMÁTICO DEL MODELO DE VARIOGRAMA
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
PEPITAS = [False, False]
MODELOS = ["Gaussian", "Gaussian"]
COLORES = ["red", "blue"]
XMAX = 11000
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

fit_params = []
R2 = []
for n, modelo in enumerate(MODELOS):
    fit_model = models[modelo](dim=2, nugget=PEPITAS[n])
    para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma[n], return_r2=True)
    fit_params.append(para)
    R2.append(r2)
    plt.plot(bin_center, gamma[n], color=COLORES[n], ls="--", marker="o")
    ax = plt.gca()
    fit_model.plot(x_max=XMAX, ax=ax, color=COLORES[n])

ax.set_ylabel("$\gamma$")
ax.set_xlabel("Distance")

for n, angulo in enumerate(ANGULOS):
    print("Ajuste dirección {}º: ".format(angulo))
    print("="*20)
    print("C: {0:.3f}".format(fit_params[n]["var"]))
    print("a: {0:.3f}".format(fit_params[n]["len_scale"]))
    print("Co: {0:.3f}".format(para["nugget"]))
    print("R2: {0:.3f}".format(R2[n]))